In [ ]:
from docuverse import SearchEngine
from docuverse import DocUVerseConfig

In [2]:
query_config = {'id_header': 'id', 'text_header': 'question', 'relevant_header': 'doc-id-list', 'answers_header': 'answers',
                'query_header': 'answers'}
